In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score,KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

### Feature Engineering

In [2]:
cd data

D:\Repozytoria Github\dw_martrix_car\data


In [3]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [4]:
SUFFIX_CAT ='__cat'
for feat in df.columns:
    if isinstance(df[feat][0],list): continue
    factorized_values = df[feat].factorize()[0]
    if SUFFIX_CAT in feat:
        df[feat] = factorized_values
    else:
        df[feat + SUFFIX_CAT] = factorized_values

In [5]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [6]:
def run_model(model, feats):
    
    X = df[feats].values
    y= df['price_value'].values
    scores = cross_val_score(model, X,y, cv=3, scoring='neg_mean_absolute_error')
    return np.mean(scores), np.std(scores)

In [7]:
xgb_params={
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
model = xgb.XGBRegressor(**xgb_params)
run_model(model, cat_feats)

(-13108.379065811214, 74.32158265003798)

In [8]:
feats=['param_napęd__cat',
'param_rok-produkcji__cat',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']


model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

(-13375.230420852275, 65.40441107118909)

In [9]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [10]:
feats=['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']


model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

(-11308.885890938496, 27.868488259630677)

In [11]:
df['param_moc'] =df['param_moc'].map(lambda x:-1 if str(x) == 'None' else int(x.split(' ')[0]))

In [12]:
feats=['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa__cat',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']


model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

(-9716.450230340148, 62.2164408321879)

In [13]:
df['param_pojemność-skokowa'] =df['param_pojemność-skokowa'].map(lambda x:-1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

feats=['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']


model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

(-9569.227198767323, 72.83561801421891)

In [ ]:
def obj_func(params):
    print('Training with params: ')
    print(params)
    
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
    
    return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


# space
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16,1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed': 0,
}


#run
best= fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)
best

Training with params:                                                                                                  
{'colsample_bytree': 0.55, 'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params:                                                                                                  
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.05, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params:                                                                                                  
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params:                                                                                                  
{'colsample_b